In [1]:
import pandas as pd
import numpy as np

#Load flight data
flights_1 = pd.read_csv('flights.csv', header=0, sep=',')

#load airline data
airlines = pd.read_csv('airlines.csv', header=0, sep=',')


#merge airline data to flight data
flights_2 = pd.merge(flights_1, 
                     airlines, 
                     how='left', 
                     left_on='AIRLINE',
                     right_on='IATA_CODE')


#load airport data
airports = pd.read_csv('airports.csv', header=0, sep=',')

#merge airport data to flight data
flights_3 = pd.merge(flights_2, 
                     airports, 
                     how='left', 
                     left_on='ORIGIN_AIRPORT', 
                     right_on='IATA_CODE')


flights_4 = pd.merge(flights_3, 
                     airports, 
                     how='left', 
                     left_on='DESTINATION_AIRPORT', 
                     right_on='IATA_CODE', 
                     suffixes=('_ORIGIN', '_DESTINATION'))

#drop duplicate columns from merging
flights = flights_4.drop(['IATA_CODE_y','IATA_CODE_x','IATA_CODE'],axis = 1)

#rename columns changed in merging
flights = flights.rename(columns = {'AIRLINE_x':'AIRLINE_CODE', 'AIRLINE_y':'AIRLINE_NAME'})

#replace NaN with 0
flights = flights.fillna(0)




/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
#Create name friendly columns
DayOfWeek = {1: 'Monday', 
             2: 'Tuesday', 
             3: 'Wednesday', 
             4: 'Thursday', 
             5: 'Friday', 
             6: 'Saturday', 
             7: 'Sunday'}

MonthName = {1: 'January', 
              2: 'February', 
              3: 'March', 
              4: 'April', 
              5: 'May', 
              6: 'June', 
              7: 'July', 
              8: 'August', 
              9: 'September', 
              10: 'October', 
              11: 'November', 
              12: 'December'}

CancellationReason = {'A': 'Airline/Carrier',
                      'B': 'Weather',
                      'C': 'National Air System',
                      'D': 'Security'}

flights['Day_Of_Week'] = flights['DAY_OF_WEEK'].map(lambda x: DayOfWeek[x])
flights['Month_Name'] = flights['MONTH'].map(lambda x: MonthName[x])
flights['Cancellation_Reason'] = flights['CANCELLATION_REASON'].map(lambda x: 'N/A' if x == 0 else CancellationReason[x])


#make dataframe for only cancelled flights
flights_c = flights[flights['CANCELLED']==1]

#make dataframe for flights with departure delays
flights_ddelays = flights[flights['DEPARTURE_DELAY'] > 0]

#make dataframe for flights with arrival delays
flights_adelays = flights[flights['ARRIVAL_DELAY'] > 0]


#makes hourly bins for departure and arrival times format "0000" 
hour_bins = np.arange(0,2500,100)
hour_bins

Hours = np.arange(0,2400,100)

time_columns = ['SCHEDULED_DEPARTURE','DEPARTURE_TIME', 'SCHEDULED_TIME', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME']

for i in time_columns:
    flights[i.lower()] = pd.cut(flights[i], 
                                        hour_bins, 
                                        labels=Hours)
    if i[0] == 'S':
        flights[i + '_SINE'] = flights[i].apply(lambda x: np.sin(2*np.pi*x/2400))
        flights[i + '_COSINE'] = flights[i].apply(lambda x: np.cos(2*np.pi*x/2400))


In [49]:
flights.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE_CODE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'AIRLINE_NAME',
       'AIRPORT_ORIGIN', 'CITY_ORIGIN', 'STATE_ORIGIN', 'COUNTRY_ORIGIN',
       'LATITUDE_ORIGIN', 'LONGITUDE_ORIGIN', 'AIRPORT_DESTINATION',
       'CITY_DESTINATION', 'STATE_DESTINATION', 'COUNTRY_DESTINATION',
       'LATITUDE_DESTINATION', 'LONGITUDE_DESTINATION', 'Day_Of_Week',
       'Month_Name', 'Cancellation_Reason', 'scheduled_departure',
       'departure_time', 'scheduled_time', 'scheduled_arrival', 'arrival_time',
   

In [68]:
# Weather data processing

import os
import datetime
directory = os.getcwd() + os.sep + 'weather_data' + os.sep
file_after = '_201501010000_201601010000.txt'
all_weather_list = []

for IATA in airports['IATA_CODE'].unique():
    airport_weather = pd.read_csv(directory + IATA + file_after, skiprows = 5)
    
    airport_weather['IATA'] = IATA
    dt = pd.to_datetime(airport_weather['valid'])
    airport_weather['MONTH'] = (dt.apply(lambda x: int(x.month))
    airport_weather['DAY'] = dt.apply(lambda x: int(x.day))
    airport_weather['HOUR'] = dt.apply(lambda x: x.hour * 100)
    airport_weather['hour'] = Categ
    
    airport_weather = airport_weather[['IATA','MONTH','DAY','HOUR','tmpf',' dwpf',' relh',' drct',' sknt',' p01i',' alti',
                                      ' mslp',' vsby',' gust',' skyc1',' skyl1']]
    
    all_weather_list.append(airport_weather)

weather = pd.concat(all_weather_list)

In [79]:
weather.dtypes
flights.dtypes

YEAR                             int64
MONTH                            int64
DAY                              int64
DAY_OF_WEEK                      int64
AIRLINE_CODE                    object
FLIGHT_NUMBER                    int64
TAIL_NUMBER                     object
ORIGIN_AIRPORT                  object
DESTINATION_AIRPORT             object
SCHEDULED_DEPARTURE              int64
DEPARTURE_TIME                 float64
DEPARTURE_DELAY                float64
TAXI_OUT                       float64
WHEELS_OFF                     float64
SCHEDULED_TIME                 float64
ELAPSED_TIME                   float64
AIR_TIME                       float64
DISTANCE                         int64
WHEELS_ON                      float64
TAXI_IN                        float64
SCHEDULED_ARRIVAL                int64
ARRIVAL_TIME                   float64
ARRIVAL_DELAY                  float64
DIVERTED                         int64
CANCELLED                        int64
CANCELLATION_REASON      

In [70]:


flights_weather = pd.merge(flights,
                           weather,
                           how = 'left',
                           left_on = ['AIRPORT_ORIGIN', 'MONTH', 'DAY','scheduled_departure'],
                           right_on = ['IATA', 'MONTH', 'DAY', 'HOUR'])


In [71]:
print(flights_weather)

         YEAR MONTH DAY  DAY_OF_WEEK AIRLINE_CODE  FLIGHT_NUMBER TAIL_NUMBER  \
0        2015     1   1            4           AS             98      N407AS   
1        2015     1   1            4           AA           2336      N3KUAA   
2        2015     1   1            4           US            840      N171US   
3        2015     1   1            4           AA            258      N3HYAA   
4        2015     1   1            4           AS            135      N527AS   
5        2015     1   1            4           DL            806      N3730B   
6        2015     1   1            4           NK            612      N635NK   
7        2015     1   1            4           US           2013      N584UW   
8        2015     1   1            4           AA           1112      N3LAAA   
9        2015     1   1            4           DL           1173      N826DN   
10       2015     1   1            4           DL           2336      N958DN   
11       2015     1   1            4    

In [50]:
from sklearn.model_selection import train_test_split


X_base = flights[['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'SCHEDULED_DEPARTURE','SCHEDULED_TIME', 'DISTANCE','SCHEDULED_ARRIVAL', 'LATITUDE_ORIGIN', 'LONGITUDE_ORIGIN',
       'LATITUDE_DESTINATION', 'LONGITUDE_DESTINATION', 'scheduled_departure', 'scheduled_time','scheduled_arrival']]

X = flights[['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK','SCHEDULED_TIME', 'DISTANCE', 'LATITUDE_ORIGIN', 'LONGITUDE_ORIGIN',
       'LATITUDE_DESTINATION', 'LONGITUDE_DESTINATION', 'SCHEDULED_DEPARTURE_SINE', 'SCHEDULED_DEPARTURE_COSINE',
            'SCHEDULED_ARRIVAL_SINE', 'SCHEDULED_ARRIVAL_COSINE']]

Y = flights['CANCELLED']

X = X.fillna(0)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, random_state=10)

In [ ]:
sklearn

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn import metrics

#Logistic Regression
model_LR = LogisticRegression()
model_LR.fit(X_train, Y_train)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [52]:

print("Area under the LR ROC curve on the test data = %.3f"
      % metrics.roc_auc_score(y_score = model_LR.predict_proba(X_test)[:,-1:], y_true = Y_test))



print("Test Data Score = %.3f" % model_LR.score(X_test ,Y_test ))


Area under the LR ROC curve on the test data = 0.687
Test Data Score = 0.984
